In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import os

# Import CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host="nv-desktop-services.apporto.com", port=31776, db_name="AAC", col_name="animals")

# Retrieve all documents and create a DataFrame
df = pd.DataFrame.from_records(db.read({}))
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug (optional)
print("DataFrame shape:", df.shape)
print(df.head())

#########################
# Dashboard Layout / View
#########################

# Load and encode the Grazioso Salvare logo
image_filename = 'grazioso_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')
logo = html.A(
    html.Img(src=f"data:image/png;base64,{encoded_image}", style={'width': '200px'}),
    href="https://www.snhu.edu", target="_blank"
)
identifier = html.H2("Andrei Shostak - CS-340 Dashboard")

# Interactive filtering options (RadioItems)
filter_options = dcc.RadioItems(
    id="filter-type",
    options=[
        {"label": "Water Rescue", "value": "Water Rescue"},
        {"label": "Mountain or Wilderness Rescue", "value": "Mountain Rescue"},
        {"label": "Disaster Rescue or Individual Tracking", "value": "Disaster Rescue"},
        {"label": "Reset", "value": "Reset"}
    ],
    value="Reset",
    labelStyle={'display': 'inline-block', 'margin-right': '20px'}
)

# Data Table
data_table = dash_table.DataTable(
    id='datatable-id',
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
    data=df.to_dict('records'),
    page_size=10,
    sort_action="native",
    row_selectable="single"
)

# Placeholders for charts
graph_div = html.Div(id='graph-id', className='col s12 m6')
map_div = html.Div(id='map-id', className='col s12 m6')

# Assemble the layout
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([logo, identifier], style={'textAlign': 'center'}),
    html.Br(),
    filter_options,
    html.Hr(),
    data_table,
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[graph_div, map_div])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter the DataTable based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    if filter_type == "Reset":
        filtered_df = pd.DataFrame.from_records(db.read({}))
    else:
        if filter_type == "Water Rescue":
            query = {"animal_type": "Dog",
                     "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                     "sex_upon_outcome": "Intact Female",
                     "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        elif filter_type == "Mountain Rescue":
            query = {"animal_type": "Dog",
                     "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                     "sex_upon_outcome": "Intact Male",
                     "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        elif filter_type == "Disaster Rescue":
            query = {"animal_type": "Dog",
                     "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                     "sex_upon_outcome": "Intact Male",
                     "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
        filtered_df = pd.DataFrame.from_records(db.read(query))
    if "_id" in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# Callback to update the second chart (pie chart of breed distribution)
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_graph(filtered_data):
    if not filtered_data:
        return html.Div("No data available.")
    temp_df = pd.DataFrame(filtered_data)
    fig = px.pie(temp_df, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)

# Callback to update the geolocation map based on selected row in the DataTable
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return html.Div("No data available for mapping.")
    dff = pd.DataFrame.from_records(viewData)
    if not selected_rows:
        row = 0
    else:
        row = selected_rows[0]
    
    required_cols = ["location_lat", "location_long", "breed", "name"]
    for col in required_cols:
        if col not in dff.columns:
            return html.Div(f"Column {col} missing from data.")
    lat = dff.loc[row, "location_lat"]
    lon = dff.loc[row, "location_long"]
    breed = str(dff.loc[row, "breed"])
    name_val = str(dff.loc[row, "name"])
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(name_val)
                        ])
                    ]
                )
            ]
        )
    ]

# Callback to update table cell styling (highlight selected columns)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{"if": {"column_id": i}, "background_color": "#D2F3FF"} for i in selected_columns]

# Run the app
app.run_server(debug=True)
